In [ ]:
!nvidia-smi

In [ ]:
!pip install --upgrade pip > /dev/null
!pip install transformers > /dev/null
!pip3 install sentencepiece > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers

from transformers import BertTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#DATADIR = "drive/MyDrive/atma10/input/"
#OUTPUTDIR = "drive/MyDrive/atma10/feature/"
DATADIR = "../input/"
OUTPUTDIR = "../feature/"

In [ ]:
train_data = pd.read_csv(DATADIR + "train.csv")
test_data = pd.read_csv(DATADIR + "test.csv")

all_df = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)

In [ ]:
class BertTranslation:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = "Helsinki-NLP/opus-mt-nl-en"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.bert_model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128
        print(self.device)


    def translation(self, sentence : str) -> np.array:
        if sentence is np.nan:
            return np.nan
        input_ids = torch.tensor([self.tokenizer.encode(sentence)], dtype=torch.long)
        input_ids = input_ids[:, :512].to(self.device)
        #print(input_ids.shape)
        output_ids = self.bert_model.generate(input_ids, max_length=512, num_beams=4, early_stopping=True)
        trased_text = self.tokenizer.decode(output_ids[0])
        return trased_text

In [ ]:
# おおよそ2~3 h 
BT = BertTranslation()

for text_col in ["long_title", "description"]:
    c = f"transed_{text_col}"
    all_df[c] = all_df[c].progress_apply(lambda x: BT.translation(x))
    all_df[["object_id", c]].to_csv(OUTPUTDIR + f"{c}.csv")